In [1]:
import pandas as pd
import numpy as np
import os
import stringdist

In [2]:
# PATH = 'C:\\Vadim\\Cisco\\Проекты\\Tax ID_v3\\SPARK_downloads'
# files = os.listdir(PATH)
# df = pd.DataFrame()
# for file in files:
#     df_part = pd.read_excel(os.path.join(PATH, file))
#     df = pd.concat([df, df_part], ignore_index=True)
# df.to_excel(os.path.join(PATH, 'result_downloads.xlsx'))

In [3]:
class Garbage_finder():
    def __init__(self, df, columns: list):
        self.df = df
        self.columns = columns
        
    def create_list_of_words(self):
        '''
        Takes all names in every cell of dataframe and creates
        an extended list of words met. Doubles are also taken
        into account
        '''
        total_list = []
        for column in self.columns:
            self.df['{}_Split'.format(column)] = self.df[column].str.split(' ')
            word_list_of_lists = self.df['{}_Split'.format(column)].tolist()
            word_list = word_list_of_lists[0]
            for list_ in word_list_of_lists[1:]:
                try:
                    word_list.extend(list_)
                except TypeError:
                    pass
            total_list.extend(word_list)
            del self.df['{}_Split'.format(column)]
        return self.df, total_list
    
    def create_bag_of_words(self, total_list, percentile=75):
        '''
        Counts frequency of every word in a list created on the previous
        stage. So called Bag of Words. Amount of trash words as counted as
        percentile of unique frequences. Defined as kwarg = 75
        '''
        from collections import Counter
        bag_of_words = Counter(total_list)
        bag_of_words_sorted = sorted(bag_of_words.items(),\
                                     key=lambda t: t[1], reverse=True)
        values = list(sorted(bag_of_words.values(), reverse=True))
        percentile = np.percentile(list(set(values)), percentile)            
        return bag_of_words_sorted, percentile
    
    def create_trash_list(self, bag_of_words, percentile):
        '''
        Using list comprehension creates a list of trash words
        '''
        trash_list = [i[0].upper() for i in bag_of_words if i[1] > percentile]
        return trash_list

In [4]:
class Cleaner():
    def __init__(self, df, columns: list, trash_list: list):
        self.df = df
        self.columns = columns
        self.trash_list = trash_list
        
    def remove_special(self, word):
        '''
        removes special symbols from string value of name
        '''
        special_symbols = ["'", '"', '“', '”', '«', 
                           '&', '?', '<', '>', ',', 
                           '.',]
        word = str(word)
        for i in special_symbols:
            if i in word:
                word = word.replace(i, '')
        return word.strip()
    
    def exclude_trash(self, word):
        '''
        excludes from name words met in trash list
        '''
        clean_word = ' '.join([i for i in word.split(' ') if i.upper() not in self.trash_list])
        return clean_word
    
    def prettify(self):
        '''
        prettifies initial dataframe
        removes special symbols
        removes trash
        '''
        for column in self.columns:
            self.df['{}_Cleaned'.format(column)] = self.df[
                column].apply(self.remove_special)
            self.df['{}_Cleaned'.format(column)] = self.df.loc[
                :, '{}_Cleaned'.format(column)].apply(self.exclude_trash)
#         self.df.sort_values(by=['Cleaned'], ascending=True, inplace=True)
#         df = df[df['Cleaned'] != '']
#         self.df = self.df.reset_index()
        return self.df
    
    def define_letter(self):
        '''
        defines first symbol of the name
        for further classification for filtering purposes
        '''
        def letter_or_number(word):
            NUMBERS = ['0','1','2','3','4','5','6','7','8','9']
            try:
                if word[0] in NUMBERS:
                    return 'digit'
                else:
                    return word[0]
            except IndexError:
                pass
        self.df['First_symbol'] = self.df['Cleaned'].apply(letter_or_number)
        return self.df
    
    def define_clean_len_words(self):
        '''
        defines len of cleaned word
        '''
        self.df['Len cleaned'] = self.df['Cleaned'].apply(lambda x: len(x.split(' ')))
        return df

In [5]:
def damerau_levenstein(row, col_previous, col_shifted):
    '''counts Damerau-Levenstein rank'''
    previous_value = str(row[col_previous])
    shifted_value = str(row[col_shifted])
    damerau_levenstein = stringdist.levenshtein(previous_value, shifted_value)
    return damerau_levenstein

In [6]:
def check_entrance(row, col_previous, col_shifted):
    '''checks whether shifted value in previous one'''
    previous_value = str(row[col_previous])
    shifted_value = str(row[col_shifted])
    if shifted_value in previous_value:
        return 'yes'
    else:
        return 'no'

In [83]:
def abbreviate(x):
    abbreviation = ''
    try:
        x_list = x.split(' ')
        for i in x_list:
            abbreviation += i[0]
    except AttributeError:
        abbreviation = x
    return abbreviation

In [120]:
def zhakkar_ratio(row, col_previous, col_shifted):
    '''counts Zhakkar ratio'''
    previous_value = str(row[col_previous]).strip().replace(' ','')
    shifted_value = str(row[col_shifted]).strip().replace(' ','')
    len_previous = len(previous_value)
    len_shifted = len(shifted_value)
    len_similar = int()
    
    from collections import Counter
    dict_previous = Counter(previous_value)
    dict_shifted = Counter(shifted_value)
    for key in list(dict_previous.keys()):
        if key in dict_shifted:
            if dict_previous[key] <= dict_shifted[key]:
                len_similar += dict_previous[key]
            else:
                len_similar += dict_shifted[key]
        else:
            pass
    
    zhakkar_ratio = (len_similar / (len_previous + len_shifted - len_similar)) * 100
    
    return int(zhakkar_ratio)

In [65]:
PATH = 'C:\Vadim\Cisco\Projects\Tax ID_v3'
file_name = 'to_validate.xlsx'

In [8]:
df = pd.read_excel(os.path.join(PATH, file_name))

In [9]:
df = df.rename(columns={'Eng name (SPARK)': 'Eng_Name_SPARK', 
                        'Short RU (SPARK)': 'Short_RU_SPARK', 
                        'Full RU (SPARK)': 'Full_RU_SPARK'})

### Validation

In [10]:
garbage = Garbage_finder(df, ['PARTY_NAME','Eng_Name_SPARK'])
df, word_list = garbage.create_list_of_words()

In [11]:
bag_of_words, percentile = garbage.create_bag_of_words(word_list, 97)
trash_list = garbage.create_trash_list(bag_of_words, percentile)
print(trash_list)

['OOO', 'AO', 'ZAO', 'LLC', 'COMPANY', 'OAO', 'LTD', 'PAO', 'OF', 'BANK', 'JSC']


In [12]:
garbage_ru = Garbage_finder(df, ['Short_RU_SPARK','Full_RU_SPARK'])
df, word_list = garbage_ru.create_list_of_words()

In [13]:
bag_of_words_ru, percentile = garbage.create_bag_of_words(word_list, 97)
trash_list_ru = garbage_ru.create_trash_list(bag_of_words_ru, percentile)
print(trash_list_ru)

['ОБЩЕСТВО', 'С', 'ОГРАНИЧЕННОЙ', 'ОТВЕТСТВЕННОСТЬЮ', 'ООО', 'АКЦИОНЕРНОЕ', 'АО', 'ПАО', 'КОМПАНИЯ', 'ПУБЛИЧНОЕ']


In [14]:
cleaner = Cleaner(df, ['PARTY_NAME', 'Eng_Name_SPARK'], trash_list)
df = cleaner.prettify()

In [15]:
cleaner_ru = Cleaner(df, ['Short_RU_SPARK','Full_RU_SPARK'], trash_list_ru)
df = cleaner_ru.prettify()

In [18]:
df.drop(['Eng_Name_SPARK', 'Short_RU_SPARK', 'Full_RU_SPARK'], axis=1, inplace=True)

In [19]:
df.head()

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_Cleaned,Eng_Name_SPARK_Cleaned,Short_RU_SPARK_Cleaned,Full_RU_SPARK_Cleaned
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,АН ПЛАНЕТА,АГЕНТСТВО НЕДВИЖИМОСТИ ПЛАНЕТА
1,7724859897,KRAFTBAU GROUP,KRAFTBAU GROUP,KRAFTBAU INVEST,КРАФТБАУ ИНВЕСТ,КРАФТБАУ ИНВЕСТ
2,7708567350,IGS OOO,IGS,IGS,ИГС,ИНВЕСТГЕОСТРОЙ
3,3435127216,ENERGOTEKHMASH OAO,ENERGOTEKHMASH,ENERGOTEKHMASH,ЭНЕРГОТЕХМАШ,ЭНЕРГОТЕХМАШ
4,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND GROUP PLC,RBS-GRUPP,РБС-ГРУПП,РБС-ГРУПП


In [20]:
df['D-L_rank_Eng'] = df.apply(
    lambda x: damerau_levenstein(
        x, col_previous='PARTY_NAME_Cleaned', col_shifted='Eng_Name_SPARK_Cleaned'), axis=1)

In [26]:
df_1 = df[df['D-L_rank_Eng'] > 1] # left after 1st cleaning iteration
len(df_1)

71768

In [34]:
df_1.head(5)

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_Cleaned,Eng_Name_SPARK_Cleaned,Short_RU_SPARK_Cleaned,Full_RU_SPARK_Cleaned,D-L_rank_Eng
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,АН ПЛАНЕТА,АГЕНТСТВО НЕДВИЖИМОСТИ ПЛАНЕТА,7
1,7724859897,KRAFTBAU GROUP,KRAFTBAU GROUP,KRAFTBAU INVEST,КРАФТБАУ ИНВЕСТ,КРАФТБАУ ИНВЕСТ,6
4,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND GROUP PLC,RBS-GRUPP,РБС-ГРУПП,РБС-ГРУПП,17
5,7731442896,HORS GROUP,HORS GROUP,KHORS,ХОРС,ХОРС,7
7,7714700709,INCHCAPE,INCHCAPE,INCHKEIP KHOLDING,ИНЧКЕЙП ХОЛДИНГ,ИНЧКЕЙП ХОЛДИНГ,12


### Validation phase_2 (partly manual)

In [41]:
garbage = Garbage_finder(df_1, ['PARTY_NAME_Cleaned','Eng_Name_SPARK_Cleaned'])
df_1, word_list = garbage.create_list_of_words()

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
bag_of_words, percentile = garbage.create_bag_of_words(word_list, 85)
trash_list = garbage.create_trash_list(bag_of_words, percentile)
print(trash_list)

['', 'KOMPANIYA', 'GROUP', 'RUS', 'OBSHESTVO', 'STOCK', 'JOINT', 'RUSSIA', 'AKCIONERNOE', 'THE', 'KB', 'I', 'AND', 'ZAVOD', 'MOSCOW', 'NM-KOMPANI', 'SERVIS', 'GRUPP', 'CENTER', 'SYSTEMS', 'SERVICE', 'LIABILITY', 'ZAKRITOE', 'INC', 'LIMITED', 'HOME', 'NAN', 'FIRMA', 'OJSC', 'IT', 'TSENTR', 'RUSSIAN', 'TELECOM', 'GLOBAL', 'S', 'GAZPROM', 'TECHNOLOGIES', 'ROSSII', 'BUSINESS', 'TD', 'REGION', 'VTB', 'INTERNATIONAL', 'CORPORATION', 'SERVICES', 'UK', 'TELEKOM']


In [46]:
trash_list_manual_eng = ['', 'KOMPANIYA', 'RUS', 'GROUP', 'OBSHESTVO', 
                        'STOCK', 'JOINT', 'AKCIONERNOE', 'THE', 'GRUPP',
                        'FIRMA', 'LIABILITY', 'ZAKRITOE', 'INC', 'LIMITED',
                        'NAN', 'OJSC', 'GLOBAL', 'TD', 'UK', 'KHOLDING'] # manual creation

In [35]:
garbage_ru = Garbage_finder(df_1, ['Short_RU_SPARK_Cleaned','Full_RU_SPARK_Cleaned'])
df_1, word_list = garbage_ru.create_list_of_words()

In [37]:
bag_of_words_ru, percentile = garbage.create_bag_of_words(word_list, 85)
trash_list_ru = garbage_ru.create_trash_list(bag_of_words_ru, percentile)
print(trash_list_ru)

['БАНК', 'ЗАКРЫТОЕ', 'ЗАО', 'РОСТЕЛЕКОМ', 'ЦЕНТР', 'И', 'РУС', '-', 'СЕРВИС', 'ТЕХНОЛОГИИ', 'ГРУПП', 'НМ-КОМПАНИ', 'ФИРМА', 'ЗАВОД', 'СИСТЕМЫ', 'ПРЕДПРИЯТИЕ', 'NAN', '', 'ГАЗПРОМ', 'ТЕЛЕКОМ', 'ОАО', 'ОТКРЫТОЕ', 'ДОМ', 'СИСТЕМС', 'ЭНД', 'КОММЕРЧЕСКИЙ', 'ГРУППА', 'АГЕНТСТВО', 'ТОРГОВЫЙ', 'СИТИБАНК', 'ГРУП', 'ИНСТИТУТ', 'КБ', 'ПЛЮС', 'ПО', 'ФИЛИАЛ', 'БИЗНЕС', 'ГЛОБАЛ', 'УПРАВЛЕНИЕ', 'КОРПОРАЦИЯ', 'МТС', 'РОССИИ', 'ВТБ', 'КОНСАЛТИНГ', 'СИ', 'УПРАВЛЯЮЩАЯ', 'СИСКО', 'ОБЩЕСТВО)', 'АЙ', 'СК', 'ВОСТОК', 'ТД']


In [47]:
trash_list_manual_ru = ['ЗАКРЫТОЕ', 'ЗАО', 'РУС', '-', 'ГРУПП', 'ФИРМА',
                       'NAN', '', 'ОАО', 'ОТКРЫТОЕ', 'ГРУППА', 'ГРУП',
                       'ФИЛИАЛ', 'ГЛОБАЛ', 'КОРПОРАЦИЯ', 'ОБЩЕСТВО)', 'ХОЛДИНГ'] # manual creation

In [51]:
cleaner = Cleaner(df_1, ['PARTY_NAME_Cleaned','Eng_Name_SPARK_Cleaned'], trash_list_manual_eng)
df_1 = cleaner.prettify()

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [52]:
cleaner_ru = Cleaner(df_1, ['Short_RU_SPARK_Cleaned','Full_RU_SPARK_Cleaned'], trash_list_manual_ru)
df_1 = cleaner_ru.prettify()

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
df_1.drop(['PARTY_NAME_Cleaned', 'Eng_Name_SPARK_Cleaned', 'Short_RU_SPARK_Cleaned',
        'Full_RU_SPARK_Cleaned', 'D-L_rank_Eng', ], axis=1, inplace=True)

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [58]:
df_1 = df_1.rename(columns={'PARTY_NAME_Cleaned_Cleaned': 'PARTY_NAME_C', 
                        'Eng_Name_SPARK_Cleaned_Cleaned': 'Eng_Name_SPARK_C', 
                        'Short_RU_SPARK_Cleaned_Cleaned': 'Short_RU_SPARK_C',
                       'Full_RU_SPARK_Cleaned_Cleaned': 'Full_RU_SPARK_C'})

In [59]:
df_1.head()

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Short_RU_SPARK_C,Full_RU_SPARK_C
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,АН ПЛАНЕТА,АГЕНТСТВО НЕДВИЖИМОСТИ ПЛАНЕТА
1,7724859897,KRAFTBAU GROUP,KRAFTBAU,KRAFTBAU INVEST,КРАФТБАУ ИНВЕСТ,КРАФТБАУ ИНВЕСТ
4,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND PLC,RBS-GRUPP,РБС-ГРУПП,РБС-ГРУПП
5,7731442896,HORS GROUP,HORS,KHORS,ХОРС,ХОРС
7,7714700709,INCHCAPE,INCHCAPE,INCHKEIP,ИНЧКЕЙП,ИНЧКЕЙП


In [61]:
df_1['D-L_rank_Eng'] = df_1.apply(
    lambda x: damerau_levenstein(
        x, col_previous='PARTY_NAME_C', col_shifted='Eng_Name_SPARK_C'), axis=1)

In [65]:
df_2 = df_1[df_1['D-L_rank_Eng'] > 1] # left after 2nd cleaning iteration
len(df_2)

69675

In [66]:
df_2.to_excel(os.path.join(PATH, 'df_2_backup.xlsx'))

### Validation Phase_3

#### Now operating with df_2

In [10]:
from transliterate import translit

In [12]:
df_2 = pd.read_excel(os.path.join(PATH, 'df_2_backup.xlsx'))

In [13]:
df_2.head()

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Short_RU_SPARK_C,Full_RU_SPARK_C,D-L_rank_Eng
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,АН ПЛАНЕТА,АГЕНТСТВО НЕДВИЖИМОСТИ ПЛАНЕТА,7
1,7724859897,KRAFTBAU GROUP,KRAFTBAU,KRAFTBAU INVEST,КРАФТБАУ ИНВЕСТ,КРАФТБАУ ИНВЕСТ,7
4,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND PLC,RBS-GRUPP,РБС-ГРУПП,РБС-ГРУПП,15
7,7714700709,INCHCAPE,INCHCAPE,INCHKEIP,ИНЧКЕЙП,ИНЧКЕЙП,4
10,5405305063,INFOTRACK,INFOTRACK,GALAKT-INFO AF,АФ ГАЛАКТ-ИНФО,АУДИТОРСКАЯ ГАЛАКТ-ИНФО,11


In [29]:
def transliterate(x):
    translated = ''
    try:
        translated = translit(x, 'ru', reversed=True).upper()
    except AttributeError:
        translated = x
    return translated

In [30]:
df_2['Full_RU_SPARK_tran'] = df_2['Full_RU_SPARK_C'].apply(lambda x: transliterate(x))

In [36]:
df_2['Len_PARTY_NAME_C'] = df_2['PARTY_NAME_C'].apply(lambda x: len(str(x)))

In [37]:
df_2.head()

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Short_RU_SPARK_C,Full_RU_SPARK_C,D-L_rank_Eng,Full_RU_SPARK_tran,Len_PARTY_NAME_C
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,АН ПЛАНЕТА,АГЕНТСТВО НЕДВИЖИМОСТИ ПЛАНЕТА,25,AGENTSTVO NEDVIZHIMOSTI PLANETA,11
1,7724859897,KRAFTBAU GROUP,KRAFTBAU,KRAFTBAU INVEST,КРАФТБАУ ИНВЕСТ,КРАФТБАУ ИНВЕСТ,7,KRAFTBAU INVEST,8
4,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND PLC,RBS-GRUPP,РБС-ГРУПП,РБС-ГРУПП,15,RBS-GRUPP,18
7,7714700709,INCHCAPE,INCHCAPE,INCHKEIP,ИНЧКЕЙП,ИНЧКЕЙП,4,INCHKEJP,8
10,5405305063,INFOTRACK,INFOTRACK,GALAKT-INFO AF,АФ ГАЛАКТ-ИНФО,АУДИТОРСКАЯ ГАЛАКТ-ИНФО,20,AUDITORSKAJA GALAKT-INFO,9


In [32]:
df_2['D-L_rank_Eng'] = df_2.apply(
    lambda x: damerau_levenstein(
        x, col_previous='PARTY_NAME_C', col_shifted='Full_RU_SPARK_tran'), axis=1)

In [46]:
df_3 = df_2[(df_2['Len_PARTY_NAME_C'] > 12) & (df_2['D-L_rank_Eng'] < 4)]

In [55]:
df_3 = df_2[~df_2.index.isin(df_3.index)] # left after 3d cleaning iteration

In [56]:
len(df_3)

66664

In [57]:
df_3.to_excel(os.path.join(PATH, 'df_3_backup.xlsx'))

### Validation Phase_4

In [67]:
df_4 = df_3[(df_3['Len_PARTY_NAME_C'] > 15) & (df_3['D-L_rank_Eng'] < 5)]

In [69]:
df_4 = df_3[~df_3.index.isin(df_4.index)]

In [70]:
len(df_4)

66033

In [71]:
df_4.to_excel(os.path.join(PATH, 'df_4_backup.xlsx'))

#### Phase 4a

In [10]:
df_4 = pd.read_excel(os.path.join(PATH, 'df_4_backup.xlsx'))

In [49]:
len(df_4[((df_4['Len_PARTY_NAME_C'] > 6) & (df_4['Len_PARTY_NAME_C'] <= 8)) & (df_4['D-L_rank_Eng'] <= 3)])

1629

In [50]:
df_5 = df_4[((df_4['Len_PARTY_NAME_C'] > 6) & (df_4['Len_PARTY_NAME_C'] <= 8)) & (df_4['D-L_rank_Eng'] <= 3)]

In [51]:
df_6 = df_4[~df_4.index.isin(df_5.index)]

In [52]:
len(df_6)

63670

In [53]:
len(df_6[((df_6['Len_PARTY_NAME_C'] > 8) & (df_6['Len_PARTY_NAME_C'] <= 10)) & (df_6['D-L_rank_Eng'] <= 4)])

2162

In [54]:
df_7 = df_6[((df_6['Len_PARTY_NAME_C'] > 8) & (df_6['Len_PARTY_NAME_C'] <= 10)) & (df_6['D-L_rank_Eng'] <= 4)]

In [55]:
df_8 = df_6[~df_6.index.isin(df_7.index)]

In [56]:
len(df_8)

61508

In [57]:
len(df_8[((df_8['Len_PARTY_NAME_C'] > 10) & (df_8['Len_PARTY_NAME_C'] <= 15)) & (df_8['D-L_rank_Eng'] <= 5)])

2896

In [58]:
df_9 = df_8[((df_8['Len_PARTY_NAME_C'] > 10) & (df_8['Len_PARTY_NAME_C'] <= 15)) & (df_8['D-L_rank_Eng'] <= 5)]

In [59]:
df_10 = df_8[~df_8.index.isin(df_9.index)]

In [60]:
len(df_10)

58612

In [61]:
len(df_10[((df_10['Len_PARTY_NAME_C'] > 15) & (df_10['Len_PARTY_NAME_C'] <= 20)) & (df_10['D-L_rank_Eng'] <= 7)])

1247

In [62]:
df_11 = df_10[((df_10['Len_PARTY_NAME_C'] > 15) & (df_10['Len_PARTY_NAME_C'] <= 20)) & (df_10['D-L_rank_Eng'] <= 7)]

In [63]:
df_12 = df_10[~df_10.index.isin(df_11.index)]

In [64]:
len(df_12)

57365

In [65]:
len(df_12[(df_12['Len_PARTY_NAME_C'] > 20) & (df_12['D-L_rank_Eng'] <= 10)])

1624

In [66]:
df_13 = df_12[(df_12['Len_PARTY_NAME_C'] > 20) & (df_12['D-L_rank_Eng'] <= 10)]

In [69]:
df_14 = df_12[~df_12.index.isin(df_13.index)]

In [70]:
len(df_14)

55741

In [71]:
df_5 = df_14

In [72]:
len(df_5)

55741

In [73]:
df_5.to_excel(os.path.join(PATH, 'df_5_backup.xlsx'))

### Validation Phase 5

In [74]:
df_5.head()

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Short_RU_SPARK_C,Full_RU_SPARK_C,D-L_rank_Eng,Full_RU_SPARK_tran,Len_PARTY_NAME_C
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,АН ПЛАНЕТА,АГЕНТСТВО НЕДВИЖИМОСТИ ПЛАНЕТА,25,AGENTSTVO NEDVIZHIMOSTI PLANETA,11
1,7724859897,KRAFTBAU GROUP,KRAFTBAU,KRAFTBAU INVEST,КРАФТБАУ ИНВЕСТ,КРАФТБАУ ИНВЕСТ,7,KRAFTBAU INVEST,8
2,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND PLC,RBS-GRUPP,РБС-ГРУПП,РБС-ГРУПП,15,RBS-GRUPP,18
3,7714700709,INCHCAPE,INCHCAPE,INCHKEIP,ИНЧКЕЙП,ИНЧКЕЙП,4,INCHKEJP,8
4,5405305063,INFOTRACK,INFOTRACK,GALAKT-INFO AF,АФ ГАЛАКТ-ИНФО,АУДИТОРСКАЯ ГАЛАКТ-ИНФО,20,AUDITORSKAJA GALAKT-INFO,9


In [80]:
df_5.drop(columns=['Short_RU_SPARK_C','Full_RU_SPARK_C', 'D-L_rank_Eng', 'Len_PARTY_NAME_C'], inplace=True)

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [81]:
df_5.head()

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Full_RU_SPARK_tran
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,AGENTSTVO NEDVIZHIMOSTI PLANETA
1,7724859897,KRAFTBAU GROUP,KRAFTBAU,KRAFTBAU INVEST,KRAFTBAU INVEST
2,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND PLC,RBS-GRUPP,RBS-GRUPP
3,7714700709,INCHCAPE,INCHCAPE,INCHKEIP,INCHKEJP
4,5405305063,INFOTRACK,INFOTRACK,GALAKT-INFO AF,AUDITORSKAJA GALAKT-INFO


In [121]:
df_5['Zhakkar_Party|Eng'] = df_5.apply(
    lambda x: zhakkar_ratio(
        x, col_previous='PARTY_NAME_C', col_shifted='Eng_Name_SPARK_C'), axis=1)

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [122]:
df_5.head(5)

,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Full_RU_SPARK_tran,Zhakkar_Party|Eng
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,AGENTSTVO NEDVIZHIMOSTI PLANETA,46
1,7724859897,KRAFTBAU GROUP,KRAFTBAU,KRAFTBAU INVEST,KRAFTBAU INVEST,57
2,7722747404,ROYAL BANK OF SCOTLAND GROUP PLC,ROYAL SCOTLAND PLC,RBS-GRUPP,RBS-GRUPP,13
3,7714700709,INCHCAPE,INCHCAPE,INCHKEIP,INCHKEJP,60
4,5405305063,INFOTRACK,INFOTRACK,GALAKT-INFO AF,AUDITORSKAJA GALAKT-INFO,46


In [127]:
len(df_5[df_5['Zhakkar_Party|Eng'] >= 55])

9834

In [150]:
df_6 = df_5[df_5['Zhakkar_Party|Eng'] < 55]

In [151]:
len(df_6)

45907

In [131]:
df_6.to_excel(os.path.join(PATH, 'df_6_backup.xlsx'))

### Validation Phase 6

In [152]:
len(df_6[df_6['Zhakkar_Party|Eng'] <= 20])

13940

In [153]:
high_risk_incorrect = df_6[df_6['Zhakkar_Party|Eng'] <= 20]

In [154]:
len(high_risk_incorrect)

13940

In [155]:
high_risk_incorrect.to_excel(os.path.join(PATH, 'high_risk_incorrect.xlsx'))

In [156]:
df_7 = df_6[df_6['Zhakkar_Party|Eng'] > 20]

In [157]:
len(df_7)

31967

In [158]:
df_7.to_excel(os.path.join(PATH, 'df_7_backup.xlsx'))

### Validation Phase 7

In [159]:
df_7.drop(columns=['Zhakkar_Party|Eng'], inplace=True)

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [162]:
df_7['Zhakkar_Party|Full'] = df_7.apply(
    lambda x: zhakkar_ratio(
        x, col_previous='PARTY_NAME_C', col_shifted='Full_RU_SPARK_tran'), axis=1)

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [164]:
df_7.drop(columns=['Zhakkar_Party|Eng'], inplace=True)

C:\Users\vpoletae\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,RU_TAX_NUMBER,PARTY_NAME,PARTY_NAME_C,Eng_Name_SPARK_C,Full_RU_SPARK_tran,Zhakkar_Party|Full
0,5407219194,PLANT METER,PLANT METER,PLANETA AN,AGENTSTVO NEDVIZHIMOSTI PLANETA,30
4,5405305063,INFOTRACK,INFOTRACK,GALAKT-INFO AF,AUDITORSKAJA GALAKT-INFO,33
6,7709430077,ADMINISTRATION,ADMINISTRATION,ADM,AKADEMIJA DETSKOGO MJUZIKLA,30
7,7713605227,LUXOFT,LUXOFT,LYUKSOFT PROFESHNL,LJUKSOFT PROFESHNL,27
10,278005403,NEFTEAVTOMATIKA PUBLIC JOINT-STOCK COMPANY,NEFTEAVTOMATIKA PUBLIC JOINT-STOCK,NEFTEAVTOMATIKA,NEFTEAVTOMATIKA,46


In [166]:
len(df_7[df_7['Zhakkar_Party|Full'] >= 55])

1065

In [168]:
df_8 = df_7[df_7['Zhakkar_Party|Full'] < 55]

In [169]:
len(df_8[df_8['Zhakkar_Party|Full'] <= 20])

3079

In [170]:
high_risk_incorrect_2 = df_8[df_8['Zhakkar_Party|Full'] <= 20]

In [171]:
high_risk_incorrect_2.to_excel(os.path.join(PATH, 'high_risk_incorrect_2.xlsx'))

In [172]:
df_8 = df_8[df_8['Zhakkar_Party|Full'] > 20]

In [173]:
len(df_8)

27823

In [174]:
df_8.to_excel(os.path.join(PATH, 'df_8_backup.xlsx'))

### Validation Phase 8

In [176]:
df_8['D-L_Party|Full'] = df_8.apply(
    lambda x: damerau_levenstein(
        x, col_previous='PARTY_NAME_C', col_shifted='Full_RU_SPARK_tran'), axis=1)

In [178]:
df_8['Zhakkar_Party|Eng'] = df_8.apply(
    lambda x: zhakkar_ratio(
        x, col_previous='PARTY_NAME_C', col_shifted='Eng_Name_SPARK_C'), axis=1)

In [179]:
df_8['D-L_Party|Eng'] = df_8.apply(
    lambda x: damerau_levenstein(
        x, col_previous='PARTY_NAME_C', col_shifted='Eng_Name_SPARK_C'), axis=1)

In [182]:
df_8['Len_PARTY_NAME_C'] = df_8['PARTY_NAME_C'].apply(lambda x: len(str(x)))

In [185]:
df_8['Q-ty_words_PARTY_NAME_C'] = df_8['PARTY_NAME_C'].apply(lambda x: len(x.split(' ')))

In [192]:
df_8['Q-ty_words_Eng_C'] = df_8['Eng_Name_SPARK_C'].apply(lambda x: len(x.split(' ')))

In [193]:
df_8['Q-ty_words_Full_C'] = df_8['Full_RU_SPARK_tran'].apply(lambda x: len(x.split(' ')))

In [195]:
df_8.to_excel(os.path.join(PATH, 'df_8_backup_adj.xlsx'))

### Validatoin phase 9 - Selenium

In [67]:
df_selenium = pd.read_excel('C:\\Vadim\\Cisco\\Projects\\Tax ID_v3\\parser\scrapped\\1.xlsx')

In [68]:
df_selenium.head(5)

,Company,Unnamed: 1
0,KRAFTBAU GROUP,"ООО ""КРАФТБАУ ГРУП"" (ИНН:7723628150)"
1,ROYAL BANK OF SCOTLAND GROUP PLC,"Королевский Банк Шотландии (ИНН 7703120329, ОГ..."
2,PUBLIC JOINT-STOCK COMPANY LETO BANK,Реквизиты ПАО «Почта Банк» | ИНН: 3232005484
3,LUXOFT,"ООО ""Люксофт Профешнл"", Москва (ИНН 7713605227..."
4,NIC SPB ETU,"АО ""НИЦ СПБ Эту"", Санкт-Петербург (ИНН 7813300..."


In [69]:
import re

In [70]:
def extract_inn(x):
    pattern = re.compile(r'(ИНН.+?)(\d+)')
    inn = int()
    try:
        inn = pattern.search(x)
        inn = int(inn.group(2))
    except (TypeError, AttributeError) as e:
        pass
    return inn

In [81]:
def extract_name(x):
    pattern = re.compile(r'(.+)(ИНН)')
    name = str()
    try:
        name = pattern.search(x)
        name = name.group(1)[:-2]
    except (TypeError, AttributeError) as e:
        pass
    return name

In [71]:
df_selenium['INN_parsed'] = df_selenium['Unnamed: 1'].apply(lambda x: extract_inn(x))

In [82]:
df_selenium['Name'] = df_selenium['Unnamed: 1'].apply(lambda x: extract_name(x))

In [84]:
df_selenium.drop(columns=['Unnamed: 1'], inplace=True)

In [85]:
df_selenium.head()

,Company,INN_parsed,Name
0,KRAFTBAU GROUP,7.723628e+09,"ООО ""КРАФТБАУ ГРУП"""
1,ROYAL BANK OF SCOTLAND GROUP PLC,7.703120e+09,Королевский Банк Шотландии
2,PUBLIC JOINT-STOCK COMPANY LETO BANK,3.232005e+09,Реквизиты ПАО «Почта Банк»
3,LUXOFT,7.713605e+09,"ООО ""Люксофт Профешнл"", Москва"
4,NIC SPB ETU,7.813301e+09,"АО ""НИЦ СПБ Эту"", Санкт-Петербург"


In [86]:
df_selenium.to_excel(os.path.join(PATH, 'df_selenium.xlsx'))